#### Import Dependencies & needed API Keys

In [1]:
import pandas as pd
import requests
import datetime
from datetime import datetime as dt
from time import sleep

from Resources.config import airNowApiKeyShane, airNowApiKeyShane2, airNowApiKeyAudelia, airNowApiKeyCenez, airNowApiKeyJoseph, airNowApiKeyJoey
from Resources.largest_combined_statistical_areas_t25 import largest_combined_statistical_areas

#### Setup counters, empty list to hold data amd API Key Variables to be used

In [2]:
# Setup Counters
counterAttempts = 0
counterAttemptsPerAPI = 0
counterAPI = 1

# Setup empty list
apiData = []

# Setup API Key Variables
airNowApiKey1 = airNowApiKeyShane
airNowApiKey2 = airNowApiKeyAudelia
airNowApiKey3 = airNowApiKeyCenez
airNowApiKey4 = airNowApiKeyJoseph
airNowApiKey5 = airNowApiKeyJoey
airNowApiKey6 = airNowApiKeyShane2

#### Create a list of dates to iterate through (first 7 days of each month, 2015 to 2019)

In [3]:
# Set Date Range to use for API
# startDate = '2015-01-01'
startDate = '2020-04-01' # <-- for testing/example; comment out this line (3), uncomment line (2) above for actual use
endDate = '2020-04-30'

# Create a list of dates (from Start to End)
dateList = pd.date_range(startDate, endDate).strftime("%Y-%m-%d").tolist()
# print(dateList)

# Create empty list to append dates within a given range to in for loop
datesToUseList = []

# Loop through dateList and append only dates that are within the range of days (of month) to the datesToUseList
for date in dateList:
    day = (datetime.datetime.strptime(date, "%Y-%m-%d")).day
    if day in range(1,8):
        datesToUseList.append(date)
#         print(f"KEEP: {date} is in range")
    else:
#         print(f"REMOVE: {date} is OUT OF RANGE")
        pass

# print(datesToUseList)
len(datesToUseList)

7

#### Setup API Call Function and with URL, API Key, and variables to pass through in the requests.get() API call

In [4]:
def apiCallLatLongHistorical(csaRank, csaFullName, csaPrimaryCity, csaPrimaryCityState, csaPopulation2018Estimate, csaPopulation2010Census, csaPrimaryCityLat, csaPrimaryCityLong, csaPrimaryCityZip, csaTimeZone, csaStandardTimeUtcOffset, csaDaylightSavingsTimeUtcOffset, csaSearchLat, csaSearchLong, csaSearchRadius, obsDate, obsTime, apiKey, apiKeyUsed, counterAttempts, counterAttemptsPerAPI, counterAPI):
    
    # Empty response list variable every time function is called
    response = []
    
    # Set API Parameters to be passed through to API requests.get
    params = {}
    params["baseURL"] = "http://www.airnowapi.org/aq/observation/latLong/historical/?format=application/json"
#     params["baseURL"] = "http://www.airnowapi.org/aq/observation/zipCode/historical/?format=application/json"
    params["lat"] = f"{csaPrimaryCityLat}"
    params["long"] = f"{csaPrimaryCityLong}"
    params["zip"] = f"{csaPrimaryCityZip}"
    params["obsDate"] = f"{obsDate}"
    params["obsTime"] = "00-0000" # f"{obsTime}"
    params["searchDistance"] = f"{csaSearchRadius}"
    params["apiKey"] = apiKey


    # Build API Request URL, passing through parameters - by LAT/LONG
    requestURL = params["baseURL"] \
                + "&latitude=" + params["lat"] \
                + "&longitude=" + params["long"] \
                + "&date=" + params["obsDate"] \
                + "T" + params["obsTime"] \
                + "&distance=" + params["searchDistance"] \
                + "&API_KEY=" + params["apiKey"]
    
#     # Build API Request URL, passing through parameters - by ZIP CODE
#     requestURL = params["baseURL"] \
#                 + "&zipCode=" + params["zip"] \
#                 + "&date=" + params["obsDate"] \
#                 + "T" + params["obsTime"] \
#                 + "&distance=" + params["searchDistance"] \
#                 + "&API_KEY=" + params["apiKey"]

    
    # Logger: Print status message
    print("--------------------------------------------------")
    print("Requesting AirNow API Data for...")
    print("     **************************************************")
    print(dt.now().strftime('%Y-%m-%d.%H.%M.%S'))
    print(f"CSA: {csaFullName} | CITY: {csaPrimaryCity}, {csaPrimaryCityState} | LAT: {params['lat']} | LONG: {params['long']} | DATE: {params['obsDate']}")
#     print(f"CSA: {csaFullName} | CITY: {csaPrimaryCity}, {csaPrimaryCityState} | ZIP: {params['zip']} | DATE: {params['obsDate']}")
    print(f"ATTEMPT:{counterAttempts} | ATTEMPT PER BATCH/API: {counterAttemptsPerAPI} | BATCH/API: {counterAPI} | API Key Used: {apiKeyUsed}")
    print("     **************************************************")
#     print(requestURL)
    
    # Set up error handling in the even there is an error in the API requests.get()
    try:
        
        # Execute requests.get, passing through built requestURL
        response = requests.get(requestURL).json()
        responseCode = requests.get(requestURL)
        
        # Results sum and response Code:
        print("     **************************************************")
        print(f"Number of results found: {len(response)}")
        print(f"Response Code: {responseCode}")
        print("     **************************************************")

        # Loop through response, appending each element (dictionary) as a new item in the apiData list
        for i in range(len(response)):
            
            # Break out the response "Category" Key value (another dictionary) into two separate values (to easily convert later to a csv, if needed)
            response[i]["AQICategoryNumber"] = response[i]["Category"]["Number"]
            response[i]["AQICategory"] = response[i]["Category"]["Name"]
            
            # Delete the "Category" Key value (another dictionary) from response since separate key: values were created and this is no longer needed
            del response[i]["Category"]
            
            # Add values brought in from the largest_combined_statistical_areas dictionary / json object to the response
            response[i]["csaRank"] = csaRank
            response[i]["csaFullName"] = csaFullName
            response[i]["csaPrimaryCity"] = csaPrimaryCity
            response[i]["csaPrimaryCityState"] = csaPrimaryCityState
            response[i]["csaPopulation2018Estimate"] = csaPopulation2018Estimate
            response[i]["csaPopulation2010Census"] = csaPopulation2010Census
            response[i]["csaPrimaryCityLat"] = csaPrimaryCityLat
            response[i]["csaPrimaryCityLong"] = csaPrimaryCityLong
            response[i]["csaPrimaryCityZip"] = csaPrimaryCityZip
          
            response[i]["csaTimeZone"] = csaTimeZone
            response[i]["csaStandardTimeUtcOffset"] = csaStandardTimeUtcOffset
            response[i]["csaDaylightSavingsTimeUtcOffset"] = csaDaylightSavingsTimeUtcOffset
            response[i]["csaSearchLat"] = csaSearchLat
            response[i]["csaSearchLong"] = csaSearchLong
            response[i]["csaSearchRadius"] = csaSearchRadius
          
            # Append the response to the apiData list
            apiData.append(response[i])

    except Exception as e:
        print(f"ERROR: Unable to perform AirNow API request for CITY: CSA: {csaFullName} | CITY: {csaPrimaryCity}, {csaPrimaryCityState} | LAT: {params['lat']} | LONG: {params['long']} | DATE: {params['obsDate']}.")
#         print(f"ERROR: Unable to perform AirNow API request for CITY: CSA: {csaFullName} | CITY: {csaPrimaryCity}, {csaPrimaryCityState} | ZIP: {params['zip']} | DATE: {params['obsDate']}.")
        print("%s" % e)
        print("--------------------------------------------------")
        pass
        

#### Loop through cities and dates, while updating variables and passing them through to the apiCallLatLongHistorical Function

In [5]:
# Loop through cities in the largest_combined_statistical_areas dictionary / json object
for csa in largest_combined_statistical_areas:
    
    # Loop through datesToUseList, calling the apiCallLatLongHistorical function, passing through variables    
    for date in datesToUseList:
    
        # Set varaiable values to pass into the apiCallLatLongHistorical function
        csaRank = csa["csa_rank"]
        csaFullName = csa["csa_name"]
        csaPrimaryCity = csa["primary_city"]
        csaPrimaryCityState = csa["primary_city_state"]
        csaPopulation2018Estimate = csa["population"]["2018_estimate"]
        csaPopulation2010Census = csa["population"]["2010_census"]
        csaPrimaryCityLat = csa["primary_city_location"]["lat"]
        csaPrimaryCityLong = csa["primary_city_location"]["long"]
        csaPrimaryCityZip = csa["primary_city_location"]["zip_code"]
        
        csaTimeZone = csa["timezone_params"]["timezone"]
        csaStandardTimeUtcOffset = csa["timezone_params"]["utc_offset"]["standard_time"]
        csaDaylightSavingsTimeUtcOffset = csa["timezone_params"]["utc_offset"]["daylight_savings_time"]
        csaSearchLat = csa["search_params"]["closest_monitoring_station"]["lat"]
        csaSearchLong = csa["search_params"]["closest_monitoring_station"]["long"]
        csaSearchRadius = csa["search_params"]["closest_monitoring_station"]["search_radius"]
        
        # Update overall attempt counter
        counterAttempts += 1
        
        # Once 250 attempts have been made for all six API Keys, reset counterAttemptsPerAPI, counterAPI and apiKey then sleep for one hour (3600 seconds)
#         if counterAttemptsPerAPI >= 250 and counterAPI >= 6:
        if counterAttemptsPerAPI >= 7 and counterAPI >= 6: # <-- for testing/example; comment out this line (30), uncomment line (29) above for actual use
            counterAttemptsPerAPI = 1
            counterAPI = 1
            apiKey = airNowApiKey1
            print("*****  250 per API for all APIs avialable reached | RESET counterAttemptsPerAPI, counterAPI, apiKey & SLEEP  *****")
            sleep(3600)
#             sleep(3)

        # Once 250 attempts have been made for the currently used API Key, update counterAPI so a new apiKey is used next time through, reset counterAttemptsPerAPI
#         elif counterAttemptsPerAPI >= 250:
        elif counterAttemptsPerAPI >= 7: # <-- for testing/example; comment out this line (40), uncomment line (39) above for actual use
            counterAPI += 1
            counterAttemptsPerAPI = 1
            print("*****  250 per current API reached | RESET counterAttemptsPerAPI; UPDATE counterAPI & apiKey  *****")
            
        else:
            counterAttemptsPerAPI += 1

        # Set apiKey variable based on the counterAPI variable
        if counterAPI == 1:
            apiKey = airNowApiKey1
            apiKeyUsed = "airNowApiKey1"
        elif counterAPI == 2:
            apiKey = airNowApiKey2
            apiKeyUsed = "airNowApiKey2"
        elif counterAPI == 3:
            apiKey = airNowApiKey3
            apiKeyUsed = "airNowApiKey3"
        elif counterAPI == 4:
            apiKey = airNowApiKey4
            apiKeyUsed = "airNowApiKey4"
        elif counterAPI == 5:
            apiKey = airNowApiKey5
            apiKeyUsed = "airNowApiKey5"
        elif counterAPI == 6:
            apiKey = airNowApiKey6
            apiKeyUsed = "airNowApiKey6"
        
        # Call the apiCallLatLongHistorical function, passing through updated variables as parameters
        apiCallLatLongHistorical(csaRank, csaFullName, csaPrimaryCity, csaPrimaryCityState, csaPopulation2018Estimate, csaPopulation2010Census, csaPrimaryCityLat, csaPrimaryCityLong, csaPrimaryCityZip, csaTimeZone, csaStandardTimeUtcOffset, csaDaylightSavingsTimeUtcOffset, csaSearchLat, csaSearchLong, csaSearchRadius, date, "00-0000", apiKey, apiKeyUsed, counterAttempts, counterAttemptsPerAPI, counterAPI)
        
        

--------------------------------------------------
Requesting AirNow API Data for...
     **************************************************
2020-04-17.16.28.20
CSA: New York-Newark, NY-NJ-CT-PA Combined Statistical Area | CITY: New York City, NY | LAT: 40.7128 | LONG: -74.006 | DATE: 2020-04-01
ATTEMPT:1 | ATTEMPT PER BATCH/API: 1 | BATCH/API: 1 | API Key Used: airNowApiKey1
     **************************************************
     **************************************************
Number of results found: 2
Response Code: <Response [200]>
     **************************************************
--------------------------------------------------
Requesting AirNow API Data for...
     **************************************************
2020-04-17.16.28.20
CSA: New York-Newark, NY-NJ-CT-PA Combined Statistical Area | CITY: New York City, NY | LAT: 40.7128 | LONG: -74.006 | DATE: 2020-04-02
ATTEMPT:2 | ATTEMPT PER BATCH/API: 2 | BATCH/API: 1 | API Key Used: airNowApiKey1
     **********

     **************************************************
Number of results found: 2
Response Code: <Response [200]>
     **************************************************
*****  250 per current API reached | RESET counterAttemptsPerAPI; UPDATE counterAPI & apiKey  *****
--------------------------------------------------
Requesting AirNow API Data for...
     **************************************************
2020-04-17.16.28.27
CSA: Chicago-Naperville, IL-IN-WI Combined Statistical Area | CITY: Chicago, IL | LAT: 41.8781 | LONG: -87.6298 | DATE: 2020-04-01
ATTEMPT:15 | ATTEMPT PER BATCH/API: 1 | BATCH/API: 3 | API Key Used: airNowApiKey3
     **************************************************
     **************************************************
Number of results found: 2
Response Code: <Response [200]>
     **************************************************
--------------------------------------------------
Requesting AirNow API Data for...
     *************************************

     **************************************************
Number of results found: 3
Response Code: <Response [200]>
     **************************************************
*****  250 per current API reached | RESET counterAttemptsPerAPI; UPDATE counterAPI & apiKey  *****
--------------------------------------------------
Requesting AirNow API Data for...
     **************************************************
2020-04-17.16.28.34
CSA: San Jose-San Francisco-Oakland, CA Combined Statistical Area | CITY: San Francisco, CA | LAT: 37.7749 | LONG: -122.4194 | DATE: 2020-04-01
ATTEMPT:29 | ATTEMPT PER BATCH/API: 1 | BATCH/API: 5 | API Key Used: airNowApiKey5
     **************************************************
     **************************************************
Number of results found: 2
Response Code: <Response [200]>
     **************************************************
--------------------------------------------------
Requesting AirNow API Data for...
     ************************

     **************************************************
Number of results found: 2
Response Code: <Response [200]>
     **************************************************
*****  250 per API for all APIs avialable reached | RESET counterAttemptsPerAPI, counterAPI, apiKey & SLEEP  *****
--------------------------------------------------
Requesting AirNow API Data for...
     **************************************************
2020-04-17.16.28.44
CSA: Dallas-Fort Worth, TX-OK Combined Statistical Area | CITY: Dallas, TX | LAT: 32.7767 | LONG: -96.797 | DATE: 2020-04-01
ATTEMPT:43 | ATTEMPT PER BATCH/API: 1 | BATCH/API: 1 | API Key Used: airNowApiKey1
     **************************************************
     **************************************************
Number of results found: 2
Response Code: <Response [200]>
     **************************************************
--------------------------------------------------
Requesting AirNow API Data for...
     ****************************

     **************************************************
Number of results found: 3
Response Code: <Response [200]>
     **************************************************
*****  250 per current API reached | RESET counterAttemptsPerAPI; UPDATE counterAPI & apiKey  *****
--------------------------------------------------
Requesting AirNow API Data for...
     **************************************************
2020-04-17.16.28.50
CSA: Houston-The Woodlands, TX Combined Statistical Area | CITY: Houston, TX | LAT: 29.7604 | LONG: -95.3698 | DATE: 2020-04-01
ATTEMPT:57 | ATTEMPT PER BATCH/API: 1 | BATCH/API: 3 | API Key Used: airNowApiKey3
     **************************************************
     **************************************************
Number of results found: 2
Response Code: <Response [200]>
     **************************************************
--------------------------------------------------
Requesting AirNow API Data for...
     ****************************************

...

     **************************************************
Number of results found: 2
Response Code: <Response [200]>
     **************************************************
*****  250 per current API reached | RESET counterAttemptsPerAPI; UPDATE counterAPI & apiKey  *****
--------------------------------------------------
Requesting AirNow API Data for...
     **************************************************
2020-04-17.16.29.20
CSA: Cleveland-Akron-Canton, OH Combined Statistical Area | CITY: Cleveland, OH | LAT: 41.4993 | LONG: -81.6944 | DATE: 2020-04-01
ATTEMPT:113 | ATTEMPT PER BATCH/API: 1 | BATCH/API: 5 | API Key Used: airNowApiKey5
     **************************************************
     **************************************************
Number of results found: 3
Response Code: <Response [200]>
     **************************************************
--------------------------------------------------
Requesting AirNow API Data for...
     ************************************

     **************************************************
Number of results found: 3
Response Code: <Response [200]>
     **************************************************
*****  250 per API for all APIs avialable reached | RESET counterAttemptsPerAPI, counterAPI, apiKey & SLEEP  *****
--------------------------------------------------
Requesting AirNow API Data for...
     **************************************************
2020-04-17.16.29.30
CSA: Portland-Vancouver-Salem, OR-WA Combined Statistical Area | CITY: Portland, OR | LAT: 45.5051 | LONG: -122.675 | DATE: 2020-04-01
ATTEMPT:127 | ATTEMPT PER BATCH/API: 1 | BATCH/API: 1 | API Key Used: airNowApiKey1
     **************************************************
     **************************************************
Number of results found: 2
Response Code: <Response [200]>
     **************************************************
--------------------------------------------------
Requesting AirNow API Data for...
     *****************

     **************************************************
Number of results found: 3
Response Code: <Response [200]>
     **************************************************
*****  250 per current API reached | RESET counterAttemptsPerAPI; UPDATE counterAPI & apiKey  *****
--------------------------------------------------
Requesting AirNow API Data for...
     **************************************************
2020-04-17.16.29.36
CSA: Charlotte-Concord, NC-SC Combined Statistical Area | CITY: Charlotte, NC | LAT: 35.2271 | LONG: -80.8431 | DATE: 2020-04-01
ATTEMPT:141 | ATTEMPT PER BATCH/API: 1 | BATCH/API: 3 | API Key Used: airNowApiKey3
     **************************************************
     **************************************************
Number of results found: 3
Response Code: <Response [200]>
     **************************************************
--------------------------------------------------
Requesting AirNow API Data for...
     **************************************

     **************************************************
Number of results found: 3
Response Code: <Response [200]>
     **************************************************
*****  250 per current API reached | RESET counterAttemptsPerAPI; UPDATE counterAPI & apiKey  *****
--------------------------------------------------
Requesting AirNow API Data for...
     **************************************************
2020-04-17.16.29.42
CSA: Pittsburgh-New Castle-Weirton, PA-OH-WV Combined Statistical Area | CITY: Pittsburgh, PA | LAT: 40.4406 | LONG: -79.9959 | DATE: 2020-04-01
ATTEMPT:155 | ATTEMPT PER BATCH/API: 1 | BATCH/API: 5 | API Key Used: airNowApiKey5
     **************************************************
     **************************************************
Number of results found: 2
Response Code: <Response [200]>
     **************************************************
--------------------------------------------------
Requesting AirNow API Data for...
     **********************

     **************************************************
Number of results found: 2
Response Code: <Response [200]>
     **************************************************
*****  250 per API for all APIs avialable reached | RESET counterAttemptsPerAPI, counterAPI, apiKey & SLEEP  *****
--------------------------------------------------
Requesting AirNow API Data for...
     **************************************************
2020-04-17.16.29.53
CSA: San Antonio-New Braunfels-Pearsall, TX Combined Statistical Area | CITY: San Antonio, TX | LAT: 29.4241 | LONG: -98.4936 | DATE: 2020-04-01
ATTEMPT:169 | ATTEMPT PER BATCH/API: 1 | BATCH/API: 1 | API Key Used: airNowApiKey1
     **************************************************
     **************************************************
Number of results found: 2
Response Code: <Response [200]>
     **************************************************
--------------------------------------------------
Requesting AirNow API Data for...
     *******

#### Check the number of records in the apiData list and visualize the data

In [6]:
len(apiData)

414

In [7]:
apiData

[{'DateObserved': '2020-04-01 ',
  'HourObserved': 0,
  'LocalTimeZone': 'EST',
  'ReportingArea': 'Newark',
  'StateCode': 'NJ',
  'Latitude': 40.7267,
  'Longitude': -74.1442,
  'ParameterName': 'OZONE',
  'AQI': 39,
  'AQICategoryNumber': 1,
  'AQICategory': 'Good',
  'csaRank': 1,
  'csaFullName': 'New York-Newark, NY-NJ-CT-PA Combined Statistical Area',
  'csaPrimaryCity': 'New York City',
  'csaPrimaryCityState': 'NY',
  'csaPopulation2018Estimate': 22679948,
  'csaPopulation2010Census': 22255491,
  'csaPrimaryCityLat': 40.7128,
  'csaPrimaryCityLong': -74.006,
  'csaPrimaryCityZip': '10001',
  'csaTimeZone': 'Eastern',
  'csaStandardTimeUtcOffset': '-05:00',
  'csaDaylightSavingsTimeUtcOffset': '-04:00',
  'csaSearchLat': 40.8419,
  'csaSearchLong': -73.8359,
  'csaSearchRadius': 25},
 {'DateObserved': '2020-04-01 ',
  'HourObserved': 0,
  'LocalTimeZone': 'EST',
  'ReportingArea': 'Newark',
  'StateCode': 'NJ',
  'Latitude': 40.7267,
  'Longitude': -74.1442,
  'ParameterName': 

#### Create DataFrame holding the values from the apiData list

In [8]:
df = pd.DataFrame(apiData)

# Visualize the DataFrame
df

,DateObserved,HourObserved,LocalTimeZone,ReportingArea,StateCode,Latitude,Longitude,ParameterName,AQI,AQICategoryNumber,...,csaPopulation2010Census,csaPrimaryCityLat,csaPrimaryCityLong,csaPrimaryCityZip,csaTimeZone,csaStandardTimeUtcOffset,csaDaylightSavingsTimeUtcOffset,csaSearchLat,csaSearchLong,csaSearchRadius
0,2020-04-01,0,EST,Newark,NJ,40.7267,-74.1442,OZONE,39,1,...,22255491,40.7128,-74.0060,10001,Eastern,-05:00,-04:00,40.8419,-73.8359,25
1,2020-04-01,0,EST,Newark,NJ,40.7267,-74.1442,PM2.5,27,1,...,22255491,40.7128,-74.0060,10001,Eastern,-05:00,-04:00,40.8419,-73.8359,25
2,2020-04-02,0,EST,Newark,NJ,40.7267,-74.1442,OZONE,39,1,...,22255491,40.7128,-74.0060,10001,Eastern,-05:00,-04:00,40.8419,-73.8359,25
3,2020-04-02,0,EST,Newark,NJ,40.7267,-74.1442,PM2.5,13,1,...,22255491,40.7128,-74.0060,10001,Eastern,-05:00,-04:00,40.8419,-73.8359,25
4,2020-04-03,0,EST,Newark,NJ,40.7267,-74.1442,OZONE,29,1,...,22255491,40.7128,-74.0060,10001,Eastern,-05:00,-04:00,40.8419,-73.8359,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,2020-04-05,0,CST,San Antonio,TX,29.4170,-98.4840,PM2.5,43,1,...,2175934,29.4241,-98.4936,78205,Central,-06:00,-05:00,29.4170,-98.4840,25
410,2020-04-06,0,CST,San Antonio,TX,29.4170,-98.4840,OZONE,41,1,...,2175934,29.4241,-98.4936,78205,Central,-06:00,-05:00,29.4170,-98.4840,25
411,2020-04-06,0,CST,San Antonio,TX,29.4170,-98.4840,PM2.5,58,2,...,2175934,29.4241,-98.4936,78205,Central,-06:00,-05:00,29.4170,-98.4840,25
412,2020-04-07,0,CST,San Antonio,TX,29.4170,-98.4840,OZONE,29,1,...,2175934,29.4241,-98.4936,78205,Central,-06:00,-05:00,29.4170,-98.4840,25


#### Set output data file filepath variable

In [9]:
output_data_filepath = "../output_data/"

#### Create variables to create the CSV file name

In [10]:
timestamp = dt.now().strftime("%Y%m%d_%H%M%S")
output_file_name = f"AirNowAPI_{timestamp}"
# print(output_file_name)

#### Convert & export DataFrame to CSV files

In [11]:
df.to_csv(f"{output_data_filepath}{output_file_name}.csv", encoding="utf-8", index= False)